In [1]:
import openpyxl
import pyomo.environ as pe
import numpy as np
import pandas as pd
from pyomo.core import expr


In [2]:
#################################################################################################################
# Objective: Read some appliance's characteristics from the excel "Info".
#
# ->Dev: Set of devices (1 a 13);
# ->Type: 0 (Not Flexibel), 1 (Power Flexibility), 2 (Time Flexibility) and 3 (Time and Power Flexibility);
# ->T_ON: Time that each load needs to be turned ON;
# ->T_OFF: Time that each load needs to be turned ON and OFF;
# ->State_init: Initial state (t=0) of each appliance: 0 (desligada) ou 1 (ligada);
# ->P_max: Maximum Power for each appliance.
# ->W_max: Maximum Weight for each appliance;
# ->W_min: Minimum Weight for each appliance;
# ->Y_max: Maximum Power percentage for each appliance;
# ->Y_min: Minimum Power percentage for each appliance;
# ->T_Hold_Power: Time needed to fix a power change for each appliance.
#
#################################################################################################################

# load the workbook
workbook = openpyxl.load_workbook('Info.xlsx')

# select the worksheet
worksheet = workbook['Lds_inputs']

# read 'Set of devices'
column = worksheet['B']
Dev = []
for cell in column:
    if cell.row == 1:
        continue
    Dev.append(cell.value)

# read 'Type of appliances'
column = worksheet['C']
Type = []
for cell in column:
    if cell.row == 1:
        continue
    Type.append(cell.value)

# read 'Time needed to be ON' 
column = worksheet['D']
T_ON = []
for cell in column:
    if cell.row == 1:
        continue
    T_ON.append(cell.value)

# read 'Time needed to be OFF' 
column = worksheet['E']
T_OFF = []
for cell in column:
    if cell.row == 1:
        continue
    T_OFF.append(cell.value)

# read 'Initial State'
column = worksheet['F']
State_init = []
for cell in column:
    if cell.row == 1:
        continue
    State_init.append(cell.value)

# read 'Maximum Power'
column = worksheet['G']
P_max = []
for cell in column:
    if cell.row == 1:
        continue
    P_max.append(cell.value)

# read 'Maximum Weight'
column = worksheet['H']
W_max = []
for cell in column:
    if cell.row == 1:
        continue
    W_max.append(cell.value)

# read 'Minimum Weight'
column = worksheet['I']
W_min = []
for cell in column:
    if cell.row == 1:
        continue
    W_min.append(cell.value)

# read 'Maximum percentage of power'
column = worksheet['J']
Y_max = []
for cell in column:
    if cell.row == 1:
        continue
    Y_max.append(cell.value)

# read 'Minimum percentage of power'
column = worksheet['K']
Y_min = []
for cell in column:
    if cell.row == 1:
        continue
    Y_min.append(cell.value)

for j in range(len(Dev)):
    Y_max [j] = Y_max [j] / 100
    Y_min [j] = Y_min [j] / 100

# read 'Time needed to fix a power change'
column = worksheet['L']
T_Hold_Power = []
for cell in column:
    if cell.row == 1:
        continue
    T_Hold_Power.append(cell.value)

# close the workbook
workbook.close()

# Print the values
print(f"Index of devices - {(Dev)}")
print(f"Type of appliances - {(Type)}")
print(f"T_ON - {(T_ON)}")
print(f"T_OFF - {(T_OFF)}")
print(f"Initial State - {(State_init)}")
print(f"Maximum Power - {(P_max)}")
print(f"Maximum Weight - {(W_max)}")
print(f"Minimum Weight - {(W_min)}")
print(f"Y_max - {(Y_max)}")
print(f"Y_min - {(Y_min)}")
print(f"T_Hold_Power - {(T_Hold_Power)}")

Index of devices - [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Type of appliances - [2, 2, 2, 2, 2, 2, 0, 0, 0, 1, 1, 1]
T_ON - [6, 15, 7, 10, 38, 32, 50, 3, 10, 40, 40, 40]
T_OFF - [4, 4, 45, 40, 65, 60, 0, 0, 0, 0, 0, 0]
Initial State - [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Maximum Power - [100, 300, 120, 120, 180, 2000, 192, 991, 1800, 250, 300, 350]
Maximum Weight - [10, 10, 10, 10, 10, 10, 10, 10, 10, 3, 3, 2]
Minimum Weight - [10, 10, 10, 10, 10, 10, 10, 10, 10, 1, 2, 1]
Y_max - [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Y_min - [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5, 0.5]
T_Hold_Power - [-1, -1, -1, -1, -1, -1, -1, -1, -1, 10, 10, 10]


In [3]:
# Sets and Constants:
T=10 # Set of indices in scheduling horizon        
P_lim = 3000 # Power limit   
Energy_price = 0.1 # Energy price   
rows = T # Auxiliary variable
cols = len(Dev) # Auxiliary variable


# On/Off variables:
X_onoff_RT_vector = [0] * len(Dev) # Status of appliance j
X_onoff_RT_matrix = [['' for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
X_onoff_opt_RT_vector = [0] * len(Dev) # Optimized Status of appliance j
X_modelo_matrix = [['' for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
x_model = [['' for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
x_opt = [['' for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
custo_matrix = [[0 for j in range(cols)] for i in range(rows)]


# Power change variables:
Y_RT_matrix = [[1 for j in range(cols)] for i in range(rows+1)] # Percentage of Power for each appliance j at time i 
Y_opt_RT_vector = [1] * len(Dev) # Percentage of Optimized Power for each appliance j


# Power variables:
P_RT = [0] * len(Dev) # Power of appliance j
P_sum=[0] * T # Sum of Power of all appliances at time i
P_sum_opt=[0] * T
P_opt_RT = [0] * len(Dev) # Optimized Power for each appliance j
P_opt_sum=[0] * T # Sum of Optimized Power of all appliances at time i


# Weight variables:
W_RT_vector = [10] * len(Dev) # Weight of appliance j
W_RT_matrix = [[10 for j in range(cols)] for i in range(rows)] # Weight of appliance j at time i
W_opt_RT_vector = [10] * len(Dev) # Optimized Weight of appliance j
W_modelo_matrix = [[10 for j in range(cols)] for i in range(rows)] # Weight of appliance j at time i
w = [[10 for j in range(cols)] for i in range(rows)]
w_opt = [[10 for j in range(cols)] for i in range(rows)]
Power = [[0 for j in range(cols)] for i in range(rows)]

# Auxiliary variables:
Optimization =0
X_onoff_before_opt_RT = [['' for j in range(cols)] for i in range(rows)]
cont_on_dataset = [0] * len(Dev)
cont_off_dataset = [0] * len(Dev)
cont_T_Hold_Power = [0] * len(Dev)
cont_on_weight = [0] * len(Dev)
cont_off_weight = [0] * len(Dev)
W_real_aux = [0] * len(Dev)
W_Power_aux = [0] * len(Dev)
custo = [[0 for j in range(cols)] for i in range(rows)]
comforto = [['' for j in range(cols)] for i in range(rows)]


#------------------------------------------------------#
# Duvido que seja necessário !!!!!!!!!!!!!!!!!!!!!!!!
weight_vector_before_optim = [['' for j in range(cols)] for i in range(rows)]
state = [0] * len(Dev) #(0=desligado, 1=ligado)
T_ON_Original = [0] * len(Dev)
T_ON_Original = T_ON.copy()
weight = [10] * len(Dev)
aux_optim = 0
force_turn_ON = 0
var_controlo = 0
T_ON_atualizado = [0] * len(Dev)
T_ON_atualizado = T_ON.copy()
control_var = [0] * len(Dev)

In [4]:
def function_dataset ():

    """ Before Optimization: """
    for i in range(T):
        for j in range(len(Dev)): 

            # Start ON 
            if (i==0 and State_init[j] == 1):     
                X_onoff_RT_matrix[i][j] = 1
                cont_on_dataset[j] += 1
                state[j]= 1
            
            # Start OFF
            elif (i==0 and State_init[j] == 0):         
                X_onoff_RT_matrix[i][j] = 0
                cont_off_dataset[j] += 1
                state[j]= 0
        
            # Loads with time flexibility
            # Loads with power flexibility 
            # Loads non-flexible
            if (i!=0):

                if state[j] == 1 and (Type[j]== 2 or Type[j]== 3):                              
                    if cont_on_dataset[j] < T_ON_atualizado[j]:                    
                        X_onoff_RT_matrix[i][j] = 1
                        cont_on_dataset[j] += 1
                        state[j]=1
                    else:                                       
                        cont_on_dataset[j] = 0
                        X_onoff_RT_matrix[i][j] = 0
                        cont_off_dataset[j] = 1
                        state[j]=0

                elif state[j] == 0 and (Type[j]== 2 or Type[j]== 3):                              
                    if cont_off_dataset[j] < T_OFF[j]:                    
                        X_onoff_RT_matrix[i][j] = 0
                        cont_off_dataset[j] += 1
                        state[j]=0
                    else:                                       
                        cont_on_dataset[j] = 1
                        X_onoff_RT_matrix[i][j] = 1
                        cont_off_dataset[j] = 0
                        state[j]=1

                elif state[j] == 0 and (Type[j]== 1 or Type[j]== 0):                          
                    if ((Type[j]== 1) and (i==4 or i==19*60 or i==20*60 or i==21*60)) or ((Type[j]== 13*60) and (i==3 or i==20*60)): 
                    #Ligar cargas com flex. em potencia (luzes) às 07:00 , 19:00 , 20:00 e 21:00; 
                    #Ligar cargas não flexiveis as 13:00 e 20:00;
                        X_onoff_RT_matrix[i][j] = 1
                        cont_on_dataset[j] += 1
                        state[j] = 1
                    else:
                        X_onoff_RT_matrix[i][j] = 0
                        state[j] = 0
                        cont_on_dataset[j] = 0

                elif state[j] == 1 and (Type[j]== 1 or Type[j]== 0): 
                    if cont_on_dataset[j] < T_ON_atualizado[j]:                    
                        cont_on_dataset[j] += 1
                        X_onoff_RT_matrix[i][j] = 1
                        state[j] = 1
                    else:
                        X_onoff_RT_matrix[i][j] = 0
                        state[j] = 0
                        cont_on_dataset[j] = 0
                       
    return X_onoff_RT_matrix





In [5]:

def function_weight (X_onoff_RT_matrix):
   
    # Before Optimization: 
    for i in range (T):

        for j in range(len(Dev)): 

            # Loads with time flexibility
            if ((Type[j]==2 or Type[j]==3)):
                if (i==0 and State_init[j] == 1): # Start ON   
                    W_RT_matrix[i][j] = 10 - (9/T_ON_atualizado[j])   # Weight
                    
                elif (i==0 and State_init[j] == 0): # Start OFF
                    W_RT_matrix[i][j] = 1 + (9/T_OFF[j])              # Weight

            # Loads non-flexible
            # Loads with power flexibility
            if (i==0 and (Type[j] == 0 or Type[j]) ) == 1:
                W_RT_matrix[i][j] = 10                                # Weight


            # Loads with time flexibility
            if i!= 0 and (Type[j]==2 or Type[j]==3):
                if X_onoff_RT_matrix[i][j] == 1 : # State ON                                                                                               
                    W_RT_matrix [i][j] =  W_RT_matrix [i-1][j] - (9/T_ON_atualizado[j] ) # Weight decreases
                if X_onoff_RT_matrix[i][j] == 0 : # State OFF 
                     W_RT_matrix [i][j] =  W_RT_matrix [i-1][j] + (9/T_OFF[j] ) # Weight increases
        
            W_RT_matrix [i][j]=round( W_RT_matrix [i][j], 3)


    return ( W_RT_matrix)




In [6]:

def function_P_sum (X_onoff_RT_matrix, P_max):
    
    for i in range(T):
        P_sum [i]= 0
    
    
    for i in range(T):
        for j in range(len(Dev)):  
            if X_onoff_RT_matrix[i][j] == 1: # State ON
                P_sum[i] = P_sum [i] + P_max[j]
    
    return(P_sum)

In [7]:

X_onoff_RT_matrix = function_dataset()
X_onoff_RT_matrix_transposta = np.transpose(X_onoff_RT_matrix)
W_RT_matrix = function_weight(X_onoff_RT_matrix)  
P_sum = function_P_sum(X_onoff_RT_matrix, P_max) 



In [8]:

model = pe.ConcreteModel()


# Set Declaration
model.loads = pe.Set(initialize=np.arange(len(Dev)))
model.Time = pe.Set(initialize=np.arange(T))

X_opt_initial_values = [[0 for j in model.loads] for i in model.Time]
W_time_opt_initial_values = [[1 for j in model.loads] for i in model.Time]
VR_initial_values = [[0 for j in model.loads] for i in model.Time]
Y_opt_initial_values = [[1 for j in model.loads] for i in model.Time]

# Parameter Declaration
model.P_lim = pe.Param(initialize=P_lim)
model.price = pe.Param(initialize=Energy_price)
model.P_max = pe.Param(model.loads, initialize=P_max)
model.State_init = pe.Param(model.loads, initialize=State_init)
model.T_OFF = pe.Param(model.loads, initialize=T_OFF)
model.T_ON = pe.Param(model.loads, initialize=T_ON)
model.Type = pe.Param(model.loads, initialize=Type)
model.W_max = pe.Param(model.loads, initialize=W_max)
model.W_min = pe.Param(model.loads, initialize=W_min)
model.Y_max = pe.Param(model.loads, initialize=Y_max)
model.Y_min = pe.Param(model.loads, initialize=Y_min)
model.X = pe.Param(model.Time, model.loads, initialize=lambda m, i, j: X_onoff_RT_matrix[i][j], mutable = True )
model.W_time = pe.Param(model.Time, model.loads, initialize=lambda m, i, j: W_RT_matrix[i][j], mutable = True)


# Variable Declaration
model.X_opt = pe.Var(model.Time, model.loads, domain=pe.Binary, initialize=lambda m, i, j: X_opt_initial_values[i][j])
model.W_time_opt = pe.Var(model.Time, model.loads, domain=pe.NonNegativeReals, initialize=lambda m, i, j:W_time_opt_initial_values [i][j])
model.W_power_opt = pe.Var(model.Time, model.loads, domain=pe.NonNegativeReals, initialize=lambda m, i, j:W_time_opt_initial_values [i][j])
model.P_rlx = pe.Var(domain=pe.NonNegativeReals, initialize = 0)
model.VR_UL = pe.Var(model.Time, model.loads, domain=pe.NonNegativeReals, initialize=lambda m, i, j: VR_initial_values[i][j])
model.VR_IL = pe.Var(model.Time, model.loads, domain=pe.NonNegativeReals, initialize=lambda m, i, j: VR_initial_values[i][j])
model.W_time_opt_aux = pe.Var(model.Time, model.loads, domain=pe.NonNegativeReals, initialize=lambda m, i, j:W_time_opt_initial_values [i][j])
model.Y_opt = pe.Var(model.Time, model.loads, domain=pe.NonNegativeReals, initialize=lambda m, i, j: Y_opt_initial_values[i][j])
model.Power = pe.Var(model.Time, model.loads, domain=pe.Binary, initialize=lambda m, i, j: X_opt_initial_values[i][j])
model.Power_aux = pe.Var(model.Time, model.loads, domain=pe.Binary, initialize=lambda m, i, j: X_opt_initial_values[i][j])


print("\nX Antes Optm\n")
X_modelo_matrix = [[model.X[i, j].value for j in model.loads] for i in model.Time]
for row in X_modelo_matrix:
    print(row)


for i in model.Time:
    for j in range(len(Dev)):
        x_model [i][j] = int(np.abs(model.X_opt[i,j].value)) # Status on/off of each loads after optimization


print("\n Model X_opt \n")
for i in range(T):
    print(x_model [i])

print("\n W\n")
w_modelo_matrix = [[model.W_time[i, j].value for j in model.loads] for i in model.Time]
for row in w_modelo_matrix:
    print(row)

print("\n W Opt \n")
w_opt_matrix = [[model.W_time_opt[i, j].value for j in model.loads] for i in model.Time]
for row in w_opt_matrix:
    print(row)

print("\n P_rlx \n")
print(model.P_rlx.value)


def constraint_one(m, i):
    return sum((m.P_max[j] * m.X_opt[i, j]*m.Y_opt[i,j]) for j in m.loads) <= m.P_lim + m.P_rlx 
model.constraint_one = pe.Constraint(model.Time, rule=constraint_one)


def _constraint_five(m, i, j):
    if m.Type[j] == 0 or m.Type[j] == 1:
        return m.X_opt[i, j] == m.X[i, j] 
    else:
        #return m.X_opt[i, j] <= m.X[i, j] 
        return pe.Constraint.Skip
model.constraint_five = pe.Constraint( model.Time, model.loads, rule=_constraint_five)


def constraint_nine_rule(m, i, j):
    if m.Type[j] == 0 or m.Type[j] == 1:
        return m.W_time_opt[i, j] == 10* m.X_opt[i, j]
    elif m.Type[j] == 2 or m.Type[j] == 3:
        if i==0:
            return m.W_time_opt[i, j] == (10 - 9/m.T_ON[j]) * m.X_opt[i, j] + (1 + 9/m.T_OFF[j]) * (1 - m.X_opt[i, j])
        elif i!= 0:
            return m.W_time_opt[i, j] == (m.W_time_opt[i-1, j] - 9/m.T_ON[j]) * m.X_opt[i, j] + (m.W_time_opt[i-1, j] + 9/m.T_OFF[j]) * (1 - m.X_opt[i, j])
        else:   
            return pe.Constraint.Skip
    else:   
        return pe.Constraint.Skip
model.constraint_nine = pe.Constraint(model.Time, model.loads, rule=constraint_nine_rule)


def constraint_a_rule(m, i, j):
    #if m.Type[j] == 2 or m.Type[j] == 0:
        #return m.W_power_opt[i, j] == 10 * m.X_opt[i, j]
    #elif m.Type[j] == 1 or m.Type[j] == 3:
    return m.W_power_opt[i, j] == ( m.W_min[j] +  ((m.W_max[j] - m.W_min[j]) / (m.Y_max[j] - m.Y_min[j] + 0.000001))*(m.Y_opt[i,j] - m.Y_min[j]) ) * m.X_opt[i, j]
    #else:   
        #return pe.Constraint.Skip
model.constraint_a = pe.Constraint(model.Time, model.loads, rule=constraint_a_rule)

# Constrait 2
def _constraint_forty(m, i, j):
        return ((m.Y_min[j] <= m.Y_opt[i,j]))
model.constraint_forty = pe.Constraint(model.Time, model.loads, rule=_constraint_forty)

# Constrait 3
def _constraint_f(m, i, j):
    return ((m.Y_opt[i,j] <= m.Y_max[j]))
model.constraint_f = pe.Constraint(model.Time, model.loads, rule=_constraint_f)


# Constrait 3
def _constraint_three(m, i, j):
    return m.W_time_opt[i,j] <= 10 + m.VR_UL[i,j]
model.constraint_three = pe.Constraint(model.Time, model.loads, rule=_constraint_three)


# Constrait 4
def _constraint_four(m, i, j):
    if m.Type[j] == 2 or m.Type[j] == 3:
        return 1 - m.VR_IL[i, j] <= m.W_time_opt[i,j] 
    else:   
        return pe.Constraint.Skip
model.constraint_four = pe.Constraint(model.Time, model.loads, rule=_constraint_four)

# Constrait 3
def _constraint_2(m, i, j):
    return m.W_time_opt_aux[i,j] == m.W_time_opt[i,j] * m.X_opt[i, j]
model.constraint_2 = pe.Constraint(model.Time, model.loads, rule=_constraint_2)


# Constrait 3
def _constraint_50(m, i, j):
    return m.Power[i,j] == (1 - m.Y_opt[i,j]) *20 * m.X_opt[i, j]
model.constraint_50 = pe.Constraint(model.Time, model.loads, rule=_constraint_50)

"""
# Constrait 3
def _constraint_40(m, i, j):
    return m.Power_aux[i,j] == m.Power[i,j]* m.X_opt[i, j]
model.constraint_40 = pe.Constraint(model.Time, model.loads, rule=_constraint_40)




# Objective function, 1º abordagem: so considerando tempo, ou seja, p=p_max sempre  --> a parte do custo vale sempre 10
def _objective_function(m):
    return sum(   m.W_time_opt_aux[i,j] - (10000000000*(m.VR_UL[i, j] + m.VR_IL[i, j] ))  - (m.price*m.P_max[j]*(10/(m.price*m.P_max[j]))*m.X_opt[i, j])  for i in m.Time for j in m.loads) - (1000000*m.P_rlx)
model.objective = pe.Objective(expr=_objective_function, sense=pe.maximize)

#- (m.price*m.P_max[j]*m.X_opt[i, j])
"""


# Objective function, 2º abordagem: tempo + potenca, o custo vale 10
def _objective_function(m):
    return sum(   m.W_time_opt_aux[i,j] - (10000000000*(m.VR_UL[i, j] + m.VR_IL[i, j] )) + m.W_power_opt[i,j] - m.Power[i,j]  for i in m.Time for j in m.loads) - (1000000*m.P_rlx)
model.objective = pe.Objective(expr=_objective_function, sense=pe.maximize)


"""
# Objective function normalizada
def _objective_function(m):
    return sum(( ((m.price*m.P_max[j]/(m.price*m.P_lim))*m.X_opt[i, j]) -((m.W_time[i,j] - 1*len(Dev))/(10*len(Dev) - 1*len(Dev)))*m.X_opt[i, j])  for i in m.Time for j in m.loads) + (1000000*m.P_rlx)
model.objective = pe.Objective(expr=_objective_function, sense=pe.minimize)



# Objective function 
def _objective_function(m):
    return sum( ((m.price*m.P_max[j])*m.X_opt[i, j]) -((m.W_time[i,j])*m.X_opt[i, j])  for i in m.Time for j in m.loads)
model.objective = pe.Objective(expr=_objective_function, sense=pe.minimize)
"""




X Antes Optm

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]

 Model X_opt 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 

'\n# Objective function normalizada\ndef _objective_function(m):\n    return sum(( ((m.price*m.P_max[j]/(m.price*m.P_lim))*m.X_opt[i, j]) -((m.W_time[i,j] - 1*len(Dev))/(10*len(Dev) - 1*len(Dev)))*m.X_opt[i, j])  for i in m.Time for j in m.loads) + (1000000*m.P_rlx)\nmodel.objective = pe.Objective(expr=_objective_function, sense=pe.minimize)\n\n\n\n# Objective function \ndef _objective_function(m):\n    return sum( ((m.price*m.P_max[j])*m.X_opt[i, j]) -((m.W_time[i,j])*m.X_opt[i, j])  for i in m.Time for j in m.loads)\nmodel.objective = pe.Objective(expr=_objective_function, sense=pe.minimize)\n'

In [9]:
###########################################################################################################
# Objetivo: Resolve problema em Real-Time
# Funções usadas: 
#                   ->function_weight (dá weight_auxs e faz optimizations dos mesmos);
#                   ->function_sum (sum as potencias de cada carga para ti);
#                   ->optimização (decide quais cargas estão ON ou OFF, maximizando o conforto);
# Input: Informação de cada carga, dataset com ON/OFF.
# Output: Optimização do dataset.
###########################################################################################################


#Resolução do problema em Real-Time
                             
        
print("\n--------------------------------Antes Opt----------------------------------------\n")

print("Dataset X\n")
          
for i in range(T):
    print(X_onoff_RT_matrix [i])

print("\n")

print("Dataset X Transposta\n")
for j in range(len(Dev)):
    print(X_onoff_RT_matrix_transposta [j])
    
print("\n")

print("Weight\n")
for i in range(T):
    print(W_RT_matrix [i])

print("\n")

print("P_sum\n")
for i in range(T):
    print(P_sum [i])


for i in model.Time:
    for j in model.loads:
        x_opt[i][j]= model.X_opt[i,j].value
        w[i][j]= model.W_time[i,j].value
       



# Solver                                                 
results = pe.SolverFactory('scip', executable='C:/Program Files/SCIPOptSuite 8.0.3/bin/scip.exe').solve(model)
#model.pprint()


print("\n-----------------------------Resultados do Modelo--------------------------------------------\n")


print("\nW_time before Opt")
for i in range(T):
    print(w[i])

print("\nW_time Opt:")
w_modelo_matrix = [[(round(model.W_time_opt[i, j].value,2)) for j in model.loads] for i in model.Time]
for row in w_modelo_matrix:
    print(row)

print("\nW_time Opt  *   x:")
wAUX_modelo_matrix = [[(round(model.W_time_opt_aux[i, j].value,2)) for j in model.loads] for i in model.Time]
for row in wAUX_modelo_matrix:
    print(row)

print("\nW_power Opt:")
wp_modelo_matrix = [[(round(model.W_power_opt[i, j].value,2)) for j in model.loads] for i in model.Time]
for row in wp_modelo_matrix:
    print(row)

print("\nX Antes Optm:")
X_modelo_matrix = [[model.X[i, j].value for j in model.loads] for i in model.Time]
for row in X_modelo_matrix:
    print(row)

for i in model.Time:
    for j in range(len(Dev)):
        x_model [i][j] = int(np.abs(model.X_opt[i,j].value)) # Status on/off of each loads after optimization

print("\nModel X_opt: ")
for i in range(T):
    print(x_model [i])

OF = model.objective()
print('\nMáximo Comforto  =', OF)

print("\nY Optm:")
Y_modelo_matrix = [[round(model.Y_opt[i, j].value,2) for j in model.loads] for i in model.Time]
for row in Y_modelo_matrix:
    print(row)

print("\nVariavel Relax UL:")
VR_UL_modelo_matrix = [[model.VR_UL[i, j].value for j in model.loads] for i in model.Time]
for row in VR_UL_modelo_matrix:
    print(row)


print("\nVariavel Relax IL:")
VR_IL_modelo_matrix = [[model.VR_IL[i, j].value for j in model.loads] for i in model.Time]
for row in VR_IL_modelo_matrix:
    print(row)

print("\nVPower:")
VR_IL_modelo_matrix = [[model.Power[i, j].value for j in model.loads] for i in model.Time]
for row in VR_IL_modelo_matrix:
    print(row)

"""
print("\n Normalizado:")

for j in range(len(Dev)):
    custo [j] = (Energy_price*P_max[j]/(Energy_price*P_lim))

for i in range(T):
    for j in range(len(Dev)):       
        comforto [i][j] = ((W_RT_matrix[i][j] - 1*len(Dev))/(10*len(Dev) - 1*len(Dev)))


print("\nCusto:")
for j in range(len(Dev)):
    print(f"j=[{j+1}] = {custo [j]}")

print("\nComforto:")
for i in range(T):
    print(f"i=[{i+1}] = {comforto [i]}")
"""

print("\n Normalizado:")

"""
for i in range(T):
    for j in range(len(Dev)):       
        comforto [i][j] = ((W_RT_matrix[i][j] ))


print("\nComforto:")
for i in range(T):
    print(f"i=[{i+1}] = {comforto [i]}")
"""

for i in range(T):
    for j in range(len(Dev)):
        custo [i][j] = (1 - model.Y_opt[i, j].value) * 20


print("\nCusto:")
for i in range(T):
    print(custo[i])

for i in range(T):
    for j in range(len(Dev)):       
        custo_matrix [i][j] = (custo [i][j]) * x_model [i][j]

print("\nCusto Matrix * x ")
for i in range(T):
    print(custo_matrix [i])



print("\nP_rlx do model:")
print(model.P_rlx.value)
#for i in range(T):
#    print(f"i=[{i+1}] = {model.P_rlx[i].value}")


"""
# TEMPO
print("\nP_max do model:")
for j in range(len(Dev)):
    print(f"j=[{j+1}] = {model.P_max [j]}")

P_sum_opt = function_P_sum(x_model , P_max)

print("\nP_sum Optm:")
for i in range(T):
    print(f"i=[{i+1}] = {P_sum_opt [i]}")

"""

"""
# TEMPO
print("\nP_max do model:")
for j in range(len(Dev)):
    print(f"j=[{j+1}] = {model.P_max [j]}")

P_sum_opt = function_P_sum(x_model , P_max)

print("\nP_sum Optm:")
for i in range(T):
    print(f"i=[{i+1}] = {P_sum_opt [i]}")

"""
# TEMPO + POTENCIA

for i in range(T):
    for j in range(len(Dev)):
        Power [i][j] = P_max [j] * model.Y_opt[i, j].value


for i in range(T):
    P_sum [i]= 0
    
    
for i in range(T):
    for j in range(len(Dev)):  
        if x_model[i][j] == 1: # State ON
            P_sum[i] = P_sum [i] + Power[i][j]

print("\nP_sum Optm:")
for i in range(T):
    print(f"i=[{i+1}] = {round(P_sum [i], 2)}")




--------------------------------Antes Opt----------------------------------------

Dataset X

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]


Dataset X Transposta

[0 0 0 0 1 1 1 1 1 1 0 0 0 0 1]
[0 0 0 0 1 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 1 1 1 1 1 1 1 1 1 1]
[0 0 0


-----------------------------Resultados do Modelo--------------------------------------------


W_time before Opt
[3.25, 3.25, 1.2, 1.225, 1.138, 1.15, 10, 10, 10, 10, 10, 10]
[5.5, 5.5, 1.4, 1.45, 1.276, 1.3, 10, 10, 10, 10, 10, 10]
[7.75, 7.75, 1.6, 1.675, 1.414, 1.45, 10, 10, 10, 10, 10, 10]
[10.0, 10.0, 1.8, 1.9, 1.552, 1.6, 10, 10, 10, 10, 10, 10]
[8.5, 9.4, 2.0, 2.125, 1.69, 1.75, 10, 10, 10, 10, 10, 10]
[7.0, 8.8, 2.2, 2.35, 1.828, 1.9, 10, 10, 10, 10, 10, 10]
[5.5, 8.2, 2.4, 2.575, 1.966, 2.05, 10, 10, 10, 10, 10, 10]
[4.0, 7.6, 2.6, 2.8, 2.104, 2.2, 10, 10, 10, 10, 10, 10]
[2.5, 7.0, 2.8, 3.025, 2.242, 2.35, 10, 10, 10, 10, 10, 10]
[1.0, 6.4, 3.0, 3.25, 2.38, 2.5, 10, 10, 10, 10, 10, 10]
[3.25, 5.8, 3.2, 3.475, 2.518, 2.65, 10, 10, 10, 10, 10, 10]
[5.5, 5.2, 3.4, 3.7, 2.656, 2.8, 10, 10, 10, 10, 10, 10]
[7.75, 4.6, 3.6, 3.925, 2.794, 2.95, 10, 10, 10, 10, 10, 10]
[10.0, 4.0, 3.8, 4.15, 2.932, 3.1, 10, 10, 10, 10, 10, 10]
[8.5, 3.4, 4.0, 4.375, 3.07, 3.25, 10, 10, 10, 10, 10, 